Ref: [Keras CNN Tutorial: Classifying Images Made Easy](https://medium.com/@prathammodi001/keras-cnn-tutorial-classifying-images-made-easy-fb55cc8892ec)

In [60]:
import tensorflow as tf 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split

In [61]:
# To check if Tensorflow is using GPU or not
print(tf.__version__)
print("GPU", "Available" if tf.config.list_physical_devices('GPU') else "Not Found")

2.16.1
GPU Not Found


### Prepare Dataset For Training <br>
`loadImages`: Function that loads all images from a specified path into a list 
<br>
`os.scandir(path)`: Gets a list of all items in the directory at `path`
<br>
`f.is_dir()`: Filters out everything that isn't a directory
<br><br>
The result is a list of paths to all subdirectories in `path`

### Define The `loadImages` function
The function loads images from the specified path and storing them to memory
<br>
It also stores the filenames of the images.

In [62]:
from os import listdir              # Used for directory operations
from PIL import Image as PImage     # PIL used for image operations
import os

In [63]:

def loadImages(path):
    # return array of images, their filenames, and labels
    subfolders = [f.path for f in os.scandir(path) if f.is_dir()]
    loadedImages = []   # Store the loaded images
    filenames = []      # Store the filenames
    labels = []         # Store the labels
    paths = []

    for folder in subfolders:
        imagesList = listdir(folder)
        label = os.path.basename(folder)    # The label is the name of folder

        for image in imagesList:
            image_path = os.path.join(folder, image)  # Full path to the image
            img = PImage.open(image_path)
            img = img.resize((224, 224))              # Resize the image
            loadedImages.append(np.array(img))        # Convert image to a numpy array
            filenames.append(image)                   # Store the filename
            labels.append(label)                      # Store the label
            paths.append(image_path)                  # Store the path

    return np.array(loadedImages), filenames, np.array(labels), paths


In [64]:
# Paths to the dataset
train_path = r"C:/UIA/CSCI 4401 FYP (1)/mvtec_anomaly_detection/tile/train"
test_path = r"C:/UIA/CSCI 4401 FYP (1)/mvtec_anomaly_detection/tile/test"
ground_truth_path = r"C:/UIA/CSCI 4401 FYP (1)/mvtec_anomaly_detection/tile/ground_truth"

In [65]:
# Load the data
train_imgs, train_filenames, train_labels, train_paths = loadImages(train_path)
test_imgs, test_filenames, test_labels, test_paths = loadImages(test_path)
ground_truth_imgs, ground_truth_filenames, ground_truth_labels, ground_truth_paths = loadImages(ground_truth_path)

# Check if the number of filenames matches the number of labels
# For training
if len(train_filenames) == len(train_labels):
    print("No missing data in training set!")
else: 
    print("There is missing data in training set!")

# For testing
if len(test_filenames) == len(test_labels):
    print("No missing data in test set!")
else: 
    print("There is missing data in test set!")

# For ground truth
if len(ground_truth_filenames) == len(ground_truth_labels):
    print("No missing data in test set!")
else: 
    print("There is missing data in ground truth set!")

No missing data in training set!
No missing data in test set!
No missing data in test set!


Function `count_files_in_subfolders` counts all files in all subdirectories of a given path.
<br>
It should give the correct count to match against the numebr of filenames returned by `loadImages()`

In [66]:
def count_files_in_subfolders(path):
    subfolders = [f.path for f in os.scandir(path) if f.is_dir()]
    count = 0

    for folder in subfolders:
        count += len(os.listdir(folder))
    return count

Check if the number of filenames matches the number of actual image files in the directories

In [67]:
# Check for train_path
if count_files_in_subfolders(train_path) == len(train_filenames):
    print("Train filenames match actual amount of files.")

else:
    print("Train filenames do not match actual amount of files, check the target directory.")

# Check for test_path
if count_files_in_subfolders(test_path) == len(test_filenames):
    print("Test filenames match actual amount of files.")

else:
    print("Test filenames do not match actual amount of files, check the target directory.")

# Check for ground_truth_path
if count_files_in_subfolders(ground_truth_path) == len(ground_truth_filenames):
    print("Ground truth filenames match actual amount of files.")

else:
    print("Ground truth filenames do not match actual amount of files, check the target directory.")

Train filenames match actual amount of files.
Test filenames match actual amount of files.
Ground truth filenames match actual amount of files.


In [68]:
# Use when filenames do not match actual amount
#print("Files found by os.listdir():", os.listdir(train_path))
#print("Filenames returned by loadImages():", train_filenames)

### Spliting Data
Splitting the training data into training set and validation set

In [69]:
from sklearn.model_selection import train_test_split

train_imgs, val_imgs, train_labels, val_labels = train_test_split(
    train_imgs, train_labels, test_size=0.2, random_state=42)

In [70]:
# Count the total number of images
total_images = count_files_in_subfolders(train_path) + count_files_in_subfolders(test_path) + count_files_in_subfolders(ground_truth_path)

print(f"Total number of images: {total_images}")

Total number of images: 431


### Initial Experiment
Experiment with 100 images

`sub_train_imgs` and `sub_train_labels` will be the new training set (80 images)
<br>
`sub_val_imgs` and `sub_val_labels` will be the new validation set (20 images)
<br><br>
It is important to shuffle data before splitting to ensure that the training and validation sets are representative of the overall distribution of the data.

In [71]:
# Select the first 100 images and labels
sub_imgs = train_imgs[:100]
sub_labels = train_labels[:100]

# Split subset into training and validation sets
sub_train_imgs, sub_val_imgs, sub_train_labels, sub_val_labels = train_test_split(
    sub_imgs, sub_labels, test_size=0.2, random_state=42
)

len(sub_train_imgs), len(sub_train_labels), len(sub_val_imgs), len(sub_val_labels)

(80, 80, 20, 20)

### Preprocessing Images
Preprocess the images into Tensors

In [72]:
import tensorflow as tf

IMG_SIZE = 224

def preprocessImage(image_path, img_size=IMG_SIZE):
    # Read the image file
    image = tf.io.read_file(image_path)

    # Turn it into numerical using 3 channels: RGB
    image = tf.image.decode_jpeg(image, channels=3)

    # Normalization: Convert 0-255 into 0-1 for each RGB
    image = tf.image.convert_image_dtype(image, tf.float32)

    # Resize to desired size: (224,224)
    image = tf.image.resize(image, size=[IMG_SIZE, IMG_SIZE])

    # Return
    return image

### Turning data into batches
Batching is a common technique used in training machine learning models, regardless of the dataset size/
<br><br>
One of the reasons in using batches:
<br>
1. Memory efficiency : You only load a small subset of data into memory each time <br><br>
2. Training speed : Allows the model to update its weights after each batch, the model can start learning before seeing all the data <br><br>
3. Generalization : Helps avoid overfitting to the training data. Shuffling the data and creating different batches for each epoch, exposes the model to a wider variety of data samples during training
***
The function `create_data_batches` creates batches of data out of image (X) and label (y) pairs
<br>
Shuffles the data if it's training data but does not if it's validation data.
<br>
It also accepts test data as input (no labels).

Create a simple function to return a tuple (image, label)
<br><br>
Takes an image file path name and the associated label, processes the image and  return a tuple of (image, label)

In [73]:
# Create function to return a tuple (image, label)
def get_image_label(image_path, label):
    image = preprocessImage(image_path)
    return image, label

In [80]:
# Create function to make batches
BATCH_SIZE = 32

def create_data_batches(X, y=None, batch_size=BATCH_SIZE, valid_data=False, test_data=False):
   # No labels for test data
   if test_data:
    print("Creating test data batches...")
    data = tf.data.Dataset.from_tensor_slices((X))

    # Only filepaths (no labels)
    data_batch = data.map(lambda x: preprocessImage(x.numpy().tobytes())).batch(BATCH_SIZE)
    return data_batch

   # If data is a valid dataset, no need to shuffle it
   elif valid_data:
      print("Creating validation data batches...")
      data = tf.data.Dataset.from_tensor_slices((X, y))   # Filepaths and labels
      data_batch = data.map(get_image_label).batch(BATCH_SIZE)
      return data_batch

   else:
      print("Creating training data batches...")
      data = tf.data.Dataset.from_tensor_slices((X, y))   # Filepaths and labels
      data_batch = data.shuffle(buffer_size=len(X)).map(get_image_label).batch(BATCH_SIZE)
      return data_batch

# Define the process_image and get_image_labels functions. 
   #  These functions are used to preprocess your images and get the imgae-label pairs, respectively

Checking the inputs to `create_data_batches`

In [81]:
print(train_imgs[:5])  # Print the first 5 elements of train_imgs
print(val_imgs[:5])    # Print the first 5 elements of val_imgs
print(test_imgs[:5])   # Print the first 5 elements of test_imgs

[[[[173 179 166]
   [177 180 163]
   [166 173 160]
   ...
   [149 142 136]
   [168 161 159]
   [168 163 161]]

  [[153 155 143]
   [161 162 148]
   [169 172 158]
   ...
   [172 166 166]
   [159 159 157]
   [126 124 118]]

  [[110 105  97]
   [121 122 115]
   [159 158 148]
   ...
   [179 179 181]
   [154 160 156]
   [103 104  97]]

  ...

  [[154 157 147]
   [160 165 153]
   [163 170 160]
   ...
   [ 86  82  76]
   [ 89  86  84]
   [ 93  91  87]]

  [[156 160 152]
   [161 165 156]
   [162 166 156]
   ...
   [ 86  82  76]
   [ 90  84  85]
   [111 108 106]]

  [[148 152 141]
   [161 165 148]
   [165 169 154]
   ...
   [ 94  90  88]
   [116 109 109]
   [140 137 135]]]


 [[[ 89  93  82]
   [ 88  88  81]
   [ 99 100  90]
   ...
   [180 188 184]
   [180 188 187]
   [152 157 153]]

  [[ 95  99  85]
   [ 91  94  83]
   [ 93  98  88]
   ...
   [182 188 187]
   [182 187 191]
   [168 175 177]]

  [[117 121 106]
   [108 112  96]
   [ 94 100  89]
   ...
   [182 191 192]
   [185 192 194]
   [181 187

Create training and validation batches

In [82]:
# Create training batches
train_data = create_data_batches(train_imgs, train_labels)

# Create validation batches
val_data = create_data_batches(val_imgs, val_labels)

# Create testing batches (if got any)
test_data = create_data_batches(test_imgs, test_labels)

Creating training data batches...


TypeError: in user code:

    File "C:\Users\sofea\AppData\Local\Temp\ipykernel_6008\734855670.py", line 3, in get_image_label  *
        image = preprocessImage(image_path)
    File "C:\Users\sofea\AppData\Local\Temp\ipykernel_6008\364219679.py", line 7, in preprocessImage  *
        image = tf.io.read_file(image_path)

    TypeError: Input 'filename' of 'ReadFile' Op has type uint8 that does not match expected type of string.
